In [1]:
import sqlite3
import pandas as pd
import requests
from datetime import datetime

print("Environment ready")


Environment ready


In [3]:
import sqlite3

conn = sqlite3.connect("weather_pipeline.db")
cursor = conn.cursor()

print("Database connected")


Database connected


In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS cities (
    city_id INTEGER PRIMARY KEY AUTOINCREMENT,
    city_name TEXT NOT NULL,
    country TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

print("Cities table created")


Cities table created


In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS weather_data (
    weather_id INTEGER PRIMARY KEY AUTOINCREMENT,
    city_id INTEGER,
    timestamp TEXT,
    temperature REAL,
    humidity INTEGER,
    pressure INTEGER,
    weather_description TEXT,
    FOREIGN KEY (city_id) REFERENCES cities(city_id)
)
""")

print("Weather data table created")


Weather data table created


In [6]:
conn.commit()
conn.close()

print("Database setup complete")


Database setup complete


In [8]:
import requests
from datetime import datetime


In [15]:
import requests
from datetime import datetime

API_KEY = "adc8656bb083a7b9f54893fca6cd8bf1"   # keep inside quotes
CITY = "Delhi"
URL = "https://api.openweathermap.org/data/2.5/weather"

params = {
    "q": CITY,
    "appid": API_KEY,
    "units": "metric"
}

response = requests.get(URL, params=params)

if response.status_code == 200:
    data = response.json()

    weather_info = {
        "city": CITY,
        "temperature": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "pressure": data["main"]["pressure"],
        "description": data["weather"][0]["description"],
        "time": datetime.now()
    }

    print("Weather Data:")
    for key, value in weather_info.items():
        print(f"{key}: {value}")

else:
    print("Error fetching data:", response.status_code)


Weather Data:
city: Delhi
temperature: 17.25
humidity: 27
pressure: 1020
description: clear sky
time: 2026-01-07 18:37:40.637796


In [16]:
import sqlite3

conn = sqlite3.connect("weather_pipeline.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS weather_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    city TEXT,
    temperature REAL,
    humidity INTEGER,
    pressure INTEGER,
    description TEXT,
    timestamp TEXT
)
""")

conn.commit()
conn.close()

print("Table created successfully")


Table created successfully


In [18]:
import requests
import sqlite3
from datetime import datetime
import pandas as pd


API_KEY = "adc8656bb083a7b9f54893fca6cd8bf1"
CITY = "Delhi"
URL = "https://api.openweathermap.org/data/2.5/weather"

params = {
    "q": CITY,
    "appid": API_KEY,
    "units": "metric"
}

response = requests.get(URL, params=params)

if response.status_code != 200:
    raise Exception("API call failed")

data = response.json()

city = CITY
temperature = data["main"]["temp"]
humidity = data["main"]["humidity"]
pressure = data["main"]["pressure"]
description = data["weather"][0]["description"]
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Weather data fetched successfully")

conn = sqlite3.connect("weather_pipeline.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS weather_data")

cursor.execute("""
CREATE TABLE weather_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    city TEXT,
    temperature REAL,
    humidity INTEGER,
    pressure INTEGER,
    description TEXT,
    timestamp TEXT
)
""")


cursor.execute("""
INSERT INTO weather_data 
(city, temperature, humidity, pressure, description, timestamp)
VALUES (?, ?, ?, ?, ?, ?)
""", (city, temperature, humidity, pressure, description, timestamp))

conn.commit()
conn.close()

print("Weather data saved to database successfully")


conn = sqlite3.connect("weather_pipeline.db")
df = pd.read_sql("SELECT * FROM weather_data", conn)
conn.close()

df


Weather data fetched successfully
Weather data saved to database successfully


,id,city,temperature,humidity,pressure,description,timestamp
0,1,Delhi,17.25,27,1020,clear sky,2026-01-07 18:43:19


In [19]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("weather_pipeline.db")

# 1. View all records
df_all = pd.read_sql("SELECT * FROM weather_data", conn)
print("ALL DATA")
display(df_all)

# 2. Average temperature
df_avg_temp = pd.read_sql("""
SELECT AVG(temperature) AS avg_temperature
FROM weather_data
""", conn)
print("AVERAGE TEMPERATURE")
display(df_avg_temp)

# 3. Highest temperature
df_max_temp = pd.read_sql("""
SELECT MAX(temperature) AS max_temperature
FROM weather_data
""", conn)
print("MAX TEMPERATURE")
display(df_max_temp)

conn.close()


ALL DATA


,id,city,temperature,humidity,pressure,description,timestamp
0,1,Delhi,17.25,27,1020,clear sky,2026-01-07 18:43:19


AVERAGE TEMPERATURE


,avg_temperature
0,17.25


MAX TEMPERATURE


,max_temperature
0,17.25


In [20]:
import requests
import sqlite3
from datetime import datetime

API_KEY = "adc8656bb083a7b9f54893fca6cd8bf1"
CITY = "Delhi"
URL = "https://api.openweathermap.org/data/2.5/weather"

params = {
    "q": CITY,
    "appid": API_KEY,
    "units": "metric"
}

response = requests.get(URL, params=params)
data = response.json()

city = CITY
temperature = data["main"]["temp"]
humidity = data["main"]["humidity"]
pressure = data["main"]["pressure"]
description = data["weather"][0]["description"]
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

conn = sqlite3.connect("weather_pipeline.db")
cursor = conn.cursor()

cursor.execute("""
INSERT INTO weather_data 
(city, temperature, humidity, pressure, description, timestamp)
VALUES (?, ?, ?, ?, ?, ?)
""", (city, temperature, humidity, pressure, description, timestamp))

conn.commit()
conn.close()

print("Weather data updated successfully")


Weather data updated successfully
